In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import cv2
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Reshape, Input, Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam

NOISE_DIM = 100
BATCH_SIZE = 64
STEPS_PER_EPOCH = 50
EPOCHS = 200
SEED = 42
WIDTH, HEIGHT, CHANNELS = 64, 64, 3  # Further reduced image size for compatibility

MAIN_DIR = "/content/drive/MyDrive/Research Project/Training"
CLASS_NAME = 'meningioma'

@tf.function
def preprocess_image(img):
    img = tf.image.resize(img, (WIDTH, HEIGHT))
    img = (tf.cast(img, tf.float32) - 127.5) / 127.5
    return img

def load_images(folder):
    class_path = os.path.join(folder, CLASS_NAME)
    dataset = tf.data.Dataset.list_files(os.path.join(class_path, "*.jpg"))
    dataset = dataset.map(lambda x: tf.io.decode_jpeg(tf.io.read_file(x), channels=3))
    dataset = dataset.map(preprocess_image)
    dataset = dataset.shuffle(1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
    return dataset

train_dataset = load_images(MAIN_DIR)
print(f"Dataset created with {len(list(train_dataset))} batches")

def build_generator():
    model = Sequential([
        Dense(8*8*256, input_dim=NOISE_DIM),
        LeakyReLU(alpha=0.2),
        Reshape((8, 8, 256)),

        Conv2DTranspose(128, (4, 4), strides=2, padding='same'),
        BatchNormalization(),
        LeakyReLU(alpha=0.2),

        Conv2DTranspose(64, (4, 4), strides=2, padding='same'),
        BatchNormalization(),
        LeakyReLU(alpha=0.2),

        Conv2DTranspose(32, (4, 4), strides=2, padding='same'),
        BatchNormalization(),
        LeakyReLU(alpha=0.2),

        Conv2D(CHANNELS, (4, 4), padding='same', activation='tanh')
    ])
    return model

def build_discriminator():
    model = Sequential([
        Conv2D(32, (3, 3), padding='same', input_shape=(WIDTH, HEIGHT, CHANNELS)),
        LeakyReLU(alpha=0.2),

        Conv2D(64, (3, 3), strides=2, padding='same'),
        BatchNormalization(),
        LeakyReLU(alpha=0.2),

        Conv2D(128, (3, 3), strides=2, padding='same'),
        BatchNormalization(),
        LeakyReLU(alpha=0.2),

        Conv2D(256, (3, 3), strides=2, padding='same'),
        BatchNormalization(),
        LeakyReLU(alpha=0.2),

        Flatten(),
        Dropout(0.3),
        Dense(1, activation="sigmoid")
    ])
    return model

class GAN(Model):
    def __init__(self, discriminator, generator):
        super(GAN, self).__init__()
        self.discriminator = discriminator
        self.generator = generator

    def compile(self, d_optimizer, g_optimizer, loss_fn):
        super(GAN, self).compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.loss_fn = loss_fn

    @tf.function
    def train_step(self, real_images):
        batch_size = tf.shape(real_images)[0]
        noise = tf.random.normal([batch_size, NOISE_DIM])

        with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
            generated_images = self.generator(noise, training=True)

            real_output = self.discriminator(real_images, training=True)
            fake_output = self.discriminator(generated_images, training=True)

            gen_loss = self.loss_fn(tf.ones_like(fake_output), fake_output)
            disc_loss = self.loss_fn(tf.ones_like(real_output), real_output) + \
                        self.loss_fn(tf.zeros_like(fake_output), fake_output)

        gradients_of_generator = gen_tape.gradient(gen_loss, self.generator.trainable_variables)
        gradients_of_discriminator = disc_tape.gradient(disc_loss, self.discriminator.trainable_variables)

        self.g_optimizer.apply_gradients(zip(gradients_of_generator, self.generator.trainable_variables))
        self.d_optimizer.apply_gradients(zip(gradients_of_discriminator, self.discriminator.trainable_variables))

        return {"d_loss": disc_loss, "g_loss": gen_loss}

# Create and compile the GAN
discriminator = build_discriminator()
generator = build_generator()
gan = GAN(discriminator, generator)
gan.compile(
    d_optimizer=Adam(0.0002, 0.5),
    g_optimizer=Adam(0.0002, 0.5),
    loss_fn=tf.keras.losses.BinaryCrossentropy(from_logits=False)
)

def save_generated_images(generated_images, folder_name="synthetic_meningioma_images"):
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

    generated_images = (generated_images * 127.5 + 127.5).numpy().astype('uint8')

    fig, axes = plt.subplots(4, 4, figsize=(20, 20))
    for i, ax in enumerate(axes.flat):
        ax.imshow(cv2.cvtColor(generated_images[i], cv2.COLOR_BGR2RGB))
        ax.axis('off')

    plt.tight_layout()
    plt.savefig(os.path.join(folder_name, "generated_images.png"), dpi=300, bbox_inches='tight')
    plt.close()

    print(f"Saved generated images in '{folder_name}' folder.")

# Training loop
for epoch in range(EPOCHS):
    print(f"Epoch {epoch+1}/{EPOCHS}")
    for real_images in tqdm(train_dataset, total=STEPS_PER_EPOCH):
        gan.train_step(real_images)

    if (epoch + 1) % 5 == 0 or epoch == 0:
        noise = tf.random.normal([16, NOISE_DIM])
        generated_images = generator(noise, training=False)
        save_generated_images(generated_images, f"epoch_{epoch+1}")



print("Training complete. Generated meningioma images saved.")

Dataset created with 21 batches
Epoch 1/200


 42%|████▏     | 21/50 [01:40<02:18,  4.79s/it]


Saved generated images in 'epoch_1' folder.
Epoch 2/200


 42%|████▏     | 21/50 [01:33<02:09,  4.47s/it]


Epoch 3/200


...

Epoch 199/200


 42%|████▏     | 21/50 [01:24<01:56,  4.02s/it]


Epoch 200/200


 42%|████▏     | 21/50 [01:26<01:59,  4.12s/it]


Saved generated images in 'epoch_200' folder.
Training complete. Generated meningioma images saved.
